In [1]:
import pandas as pd
import numpy as np
import time
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input

In [2]:
data= pd.read_csv(r"C:\Users\hsynt\Downloads\Selcuk Abi\Selcuk Abi\VeriSeti2.csv")

In [3]:
# Özellik ve Hedef Ayrımı
X = data.drop(columns=['Target_10', 'adjclose'])
y = data['Target_10']

# Zaman Serisi Formatına Getirme
def create_sequences(data, target, seq_length):
    sequences, targets = [], []
    for i in range(len(data) - seq_length):
        sequences.append(data[i:i + seq_length])
        targets.append(target[i + seq_length])
    return np.array(sequences), np.array(targets)

SEQ_LENGTH = 10
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_seq, y_seq = create_sequences(X_scaled, y.values, SEQ_LENGTH)

# PCA ve LDA Veri Hazırlığı
X_flat = X_seq.reshape(X_seq.shape[0], -1)  # Düzleştirilmiş veri

pca = PCA(n_components=10)
X_pca = pca.fit_transform(X_flat)

lda = LDA(n_components=1)
X_lda = lda.fit_transform(X_flat, y_seq)

# LSTM Modeli Tanımlama (Özellik Çıkartma için)
def build_lstm_feature_extractor(input_shape):
    model = Sequential([
        Input(shape=input_shape),
        LSTM(64, activation='relu', return_sequences=False),
        Dropout(0.2),
        Dense(32, activation='relu')  # Ara katman: Özellik Çıkarıcı
    ])
    return model

# K-Fold Cross Validation
kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

# Eğitim ve Test Sürelerini Hesaplama
results = []

for reduction, X_data in zip(
    ['No PCA/LDA', 'PCA', 'LDA'],
    [X_seq, X_pca, X_lda]
):
    fold_num = 1
    for train_idx, test_idx in kfold.split(X_data, y_seq):
        # Eğitim ve Test Bölünmesi
        X_train, X_test = X_data[train_idx], X_data[test_idx]
        y_train, y_test = y_seq[train_idx], y_seq[test_idx]

        if reduction == 'No PCA/LDA':
            X_train = X_train.reshape(X_train.shape[0], SEQ_LENGTH, -1)
            X_test = X_test.reshape(X_test.shape[0], SEQ_LENGTH, -1)

            # LSTM Modeli ile Özellik Çıkartma
            lstm_model = build_lstm_feature_extractor(X_train.shape[1:])
            start_time = time.time()
            X_train_features = lstm_model.predict(X_train)
            X_test_features = lstm_model.predict(X_test)
            lstm_train_time = time.time() - start_time  # LSTM Eğitim Süresi

        else:
            # PCA ve LDA için düzleştirilmiş veriler kullanılır
            start_time = time.time()
            X_train_features = X_train  # Doğrudan PCA/LDA özellikleri
            X_test_features = X_test
            lstm_train_time = 0  # LSTM eğitim süresi yok

        # Random Forest Modeli
        rf_model = RandomForestClassifier(n_estimators=200, random_state=42)
        
        # Random Forest Eğitim Süresi
        start_time = time.time()
        rf_model.fit(X_train_features, y_train)
        rf_train_time = time.time() - start_time

        # Random Forest Test Süresi
        start_time = time.time()
        rf_model.predict(X_test_features)
        rf_test_time = time.time() - start_time

        total_train_time = lstm_train_time + rf_train_time

        results.append({
            'Reduction': f"{reduction} (Fold {fold_num})",
            'Model': 'LSTM + Random Forest',
            'Train Time (s)': total_train_time,
            'Test Time (s)': rf_test_time
        })

        fold_num += 1

# Performans Sonuçlarını Görselleştirme
results_df = pd.DataFrame(results)

# Eğitim ve Test Süreleri Tablosu
print("Eğitim ve Test Süreleri Tablosu:")
print(results_df)

# Sonuçları Kaydetme
results_df.to_csv("lstm_rf_kfold_train_test_times.csv", index=False)
print("\nSonuçlar 'lstm_rf_kfold_train_test_times.csv' dosyasına kaydedildi.")

1041/1041 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
521/521 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
1041/1041 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step
521/521 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
1041/1041 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
521/521 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step
Eğitim ve Test Süreleri Tablosu:
             Reduction                 Model  Train Time (s)  Test Time (s)
0  No PCA/LDA (Fold 1)  LSTM + Random Forest       22.029397       0.352021
1  No PCA/LDA (Fold 2)  LSTM + Random Forest       22.835051       0.342881
2  No PCA/LDA (Fold 3)  LSTM + Random Forest       19.965538       0.370633
3         PCA (Fold 1)  LSTM + Random Forest       21.499062       0.399276
4         PCA (Fold 2)  LSTM + Random Forest       21.754650       0.371303
5         PCA (Fold 3)  LSTM + Random Forest       21.322821       0.370142
6         LDA (Fold 1)  LSTM + Random Forest        9.141084       0.371427
7         LDA (Fold 2)  LSTM + Random Forest        8.633897       0.368167
8         LDA (Fold 3)  LSTM + 